# libraries


In [11]:
     
import serial
import time
import cv2
import numpy as np

# modbus to regular


In [2]:
import struct
import time
from pymodbus.client import ModbusSerialClient as ModbusClient

# הגדרות החיבור - ניתן לשינוי
PORT = "COM3"  # שם הפורט - נסה גם COM5 אם לא עובד
BAUDRATE = 115200  # הורדנו מ230400 לערך יותר סטנדרטי
TIMEOUT = 2  # זמן מקסימלי להמתנה לתשובה
SLAVE_ID = 1  # כתובת המכשיר
REGISTER_ADDRESS = 0  # כתובת הרגיסטר
REGISTER_COUNT = 2  # מספר רגיסטרים לקריאה

def create_modbus_client():
    """
    יוצר ומחזיר לקוח Modbus RTU עם הגדרות מותאמות
    """
    client = ModbusClient(
        port=PORT,
        baudrate=BAUDRATE,
        timeout=TIMEOUT,
        parity='N',  # No parity
        stopbits=1,
        bytesize=8,
        strict=False  # פחות מחמיר על שגיאות
    )
    return client

def read_modbus_registers(slave_id=SLAVE_ID, register_address=REGISTER_ADDRESS, count=REGISTER_COUNT):
    """
    קורא רגיסטרים בפרוטוקול MODBUS RTU ומחזיר את הנתונים.
    מטפל בשגיאות ומנסה מספר פעמים במקרה של כישלון.
    """
    client = create_modbus_client()
    
    # נסיון התחברות עם מספר ניסיונות
    max_attempts = 3
    for attempt in range(max_attempts):
        try:
            print(f"🔄 Attempting connection to {PORT} (attempt {attempt + 1}/{max_attempts})...")
            
            if client.connect():
                print(f"✅ Successfully connected to {PORT}")
                break
            else:
                print(f"❌ Failed to connect to {PORT}")
                if attempt < max_attempts - 1:
                    time.sleep(1)  # המתנה קצרה לפני ניסיון נוסף
                continue
                
        except Exception as e:
            print(f"❌ Connection error: {e}")
            if attempt < max_attempts - 1:
                time.sleep(1)
            continue
    else:
        print("❌ Failed to connect after all attempts")
        return None

    try:
        # קריאת רגיסטרים
        print(f"📖 Reading {count} registers from address {register_address}, slave {slave_id}")
        response = client.read_input_registers(
            address=register_address, 
            count=count, 
            slave=slave_id
        )
        
        if response.isError():
            print(f"❌ Modbus Error: {response}")
            return None
            
        print(f"✅ Successfully read {len(response.registers)} registers")
        return response.registers
        
    except Exception as e:
        print(f"❌ Read error: {e}")
        return None
        
    finally:
        try:
            client.close()
            print("🔌 Connection closed")
        except:
            pass

def convert_to_a2d(modbus_values, v_ref=5.0, resolution=16):
    """
    ממיר ערכים שהתקבלו ממודבוס לערכים אנלוגיים (A2D).
    """
    if not modbus_values:
        return []
        
    max_value = 2**resolution - 1  # 65535 עבור 16 ביט
    a2d_values = []
    
    for val in modbus_values:
        # בדיקת תקינות הערך
        if 0 <= val <= max_value:
            voltage = (val / max_value) * v_ref
            a2d_values.append(voltage)
        else:
            print(f"⚠️ Invalid value detected: {val} (expected 0-{max_value})")
            a2d_values.append(0.0)
    
    return a2d_values

def test_modbus_communication():
    """
    פונקציה לבדיקת תקשורת Modbus
    """
    print("=" * 50)
    print("🧪 Testing Modbus Communication")
    print("=" * 50)
    print(f"Port: {PORT}")
    print(f"Baudrate: {BAUDRATE}")
    print(f"Slave ID: {SLAVE_ID}")
    print(f"Register Address: {REGISTER_ADDRESS}")
    print(f"Register Count: {REGISTER_COUNT}")
    print("-" * 50)
    
    # קריאת נתונים
    modbus_data = read_modbus_registers()
    
    if modbus_data:
        print(f"📊 Raw Modbus Data: {modbus_data}")
        
        # המרה לערכי A2D
        a2d_values = convert_to_a2d(modbus_data)
        print(f"🔢 Converted A2D Values (V): {[f'{v:.3f}' for v in a2d_values]}")
        
        # הצגת מידע נוסף
        for i, (raw, voltage) in enumerate(zip(modbus_data, a2d_values)):
            percentage = (voltage / 5.0) * 100
            print(f"   Register {i}: {raw} → {voltage:.3f}V ({percentage:.1f}%)")
            
        return True
    else:
        print("⚠️ Failed to read data. Check:")
        print("   • COM port number and connection")
        print("   • Baud rate settings")
        print("   • Device slave ID")
        print("   • Register address")
        print("   • Physical wiring")
        return False

# הפעלה ידנית - הסר את התגובה כדי להפעיל
# test_modbus_communication()

In [4]:
# Modbus Configuration and Testing

# Try different configurations if the default doesn't work
MODBUS_CONFIGS = [
    {"port": "COM3", "baudrate": 115200, "parity": 'N'},
    {"port": "COM11", "baudrate": 115200, "parity": 'N'},
    {"port": "COM5", "baudrate": 115200, "parity": 'N'},
    {"port": "COM3", "baudrate": 230400, "parity": 'N'},
    {"port": "COM11", "baudrate": 230400, "parity": 'N'},
    {"port": "COM5", "baudrate": 230400, "parity": 'N'},
    {"port": "COM3", "baudrate": 9600, "parity": 'N'},
    {"port": "COM11", "baudrate": 9600, "parity": 'N'},
    {"port": "COM5", "baudrate": 9600, "parity": 'N'},
]

def try_different_configs():
    """
    נסה תצורות שונות של Modbus עד שאחת עובדת
    """
    global PORT, BAUDRATE
    
    for i, config in enumerate(MODBUS_CONFIGS):
        print(f"\n🔧 Trying configuration {i+1}/{len(MODBUS_CONFIGS)}")
        print(f"   Port: {config['port']}, Baudrate: {config['baudrate']}")
        
        # עדכון הגדרות
        PORT = config['port']
        BAUDRATE = config['baudrate']
        
        # ניסיון קריאה
        if test_modbus_communication():
            print(f"✅ Success with configuration: {config}")
            return True
        
        print(f"❌ Configuration {i+1} failed")
    
    print("\n⚠️ All configurations failed. Check hardware connection.")
    return False

def continuous_monitoring(duration_seconds=30):
    """
    ניטור רצוף של נתוני Modbus
    """
    print(f"🔄 Starting continuous monitoring for {duration_seconds} seconds...")
    print("Press Ctrl+C to stop early")
    
    start_time = time.time()
    reading_count = 0
    
    try:
        while time.time() - start_time < duration_seconds:
            modbus_data = read_modbus_registers()
            
            if modbus_data:
                a2d_values = convert_to_a2d(modbus_data)
                reading_count += 1
                
                print(f"Reading {reading_count}: {[f'{v:.3f}V' for v in a2d_values]}")
            else:
                print("❌ Failed to read data")
            
            time.sleep(1)  # קריאה כל שנייה
            
    except KeyboardInterrupt:
        print("\n⏹️ Monitoring stopped by user")
    
    print(f"📊 Completed {reading_count} successful readings")

# 🧪 QUICK TEST - Uncomment ONE of these lines to test:
test_modbus_communication()    # Single test with current settings
# try_different_configs()      # Try multiple configurations automatically
# continuous_monitoring(10)    # Continuous monitoring for 10 seconds

🧪 Testing Modbus Communication
Port: COM3
Baudrate: 115200
Slave ID: 1
Register Address: 0
Register Count: 2
--------------------------------------------------


TypeError: ModbusSerialClient.__init__() got an unexpected keyword argument 'strict'

# main - GAS

In [23]:
# Optical Bench Alignment Testing System
import serial
import time
import wave
import cv2
import numpy as np
import pandas as pd
from threading import Thread
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
from datetime import datetime

# הגדרות תצוגה
width, height = 600, 700
font = cv2.FONT_HERSHEY_SIMPLEX

def process_channel(channel_bytes):
    swapped_bytes = channel_bytes[2:3] + channel_bytes[1:2] + channel_bytes[0:1]
    value = int.from_bytes(swapped_bytes, byteorder='big')
    return value

def process_packet(packet):
    data_bytes = packet[4:]
    channels = [data_bytes[i:i+3] for i in range(0, 12, 3)]
    return [process_channel(ch) for ch in channels]

def update_live_plot(angles, ch1_norm, ch2_norm, ratio_values, selected_channels, test_mode, current_axis, fig=None):
    # Use the existing figure or create one if it doesn't exist
    if fig is None:
        fig = plt.figure(figsize=(12, 8))
    
    # Clear the figure completely
    fig.clear()
    
    if test_mode == "single":
        ax1 = fig.add_subplot(2, 1, 1)
        if len(ch1_norm) > 0:
            ax1.plot(angles, ch1_norm, 'o-', label=f'Ch{selected_channels[0]} (normalized)', color='blue')
        if len(ch2_norm) > 0:
            ax1.plot(angles, ch2_norm, 'o-', label=f'Ch{selected_channels[1]} (normalized)', color='red')
        ax1.set_xlabel('Angle (degrees)')
        ax1.set_ylabel('Normalized Signal')
        ax1.set_title('Channel Alignment Test')
        ax1.legend()
        ax1.grid(True)
        
        ax2 = fig.add_subplot(2, 1, 2)
        if len(ratio_values) > 0:
            ax2.plot(angles, ratio_values, 'o-', label=f'Ratio Ch{selected_channels[0]}/Ch{selected_channels[1]}', color='green')
        ax2.set_xlabel('Angle (degrees)')
        ax2.set_ylabel('Ratio')
        ax2.set_title('Channel Ratio')
        ax2.legend()
        ax2.grid(True)
    
    else:  # dual axis
        ax1 = fig.add_subplot(2, 1, 1)
        x_indices = [i for i, axis in enumerate(current_axis) if axis == 'X']
        y_indices = [i for i, axis in enumerate(current_axis) if axis == 'Y']
        
        if x_indices:
            x_angles = [angles[i] for i in x_indices]
            x_ch1 = [ch1_norm[i] for i in x_indices]
            x_ch2 = [ch2_norm[i] for i in x_indices]
            ax1.plot(x_angles, x_ch1, 'o-', label=f'X-axis Ch{selected_channels[0]}', color='blue')
            ax1.plot(x_angles, x_ch2, 'o-', label=f'X-axis Ch{selected_channels[1]}', color='red')
        
        if y_indices:
            y_angles = [angles[i] for i in y_indices]
            y_ch1 = [ch1_norm[i] for i in y_indices]
            y_ch2 = [ch2_norm[i] for i in y_indices]
            ax1.plot(y_angles, y_ch1, 's-', label=f'Y-axis Ch{selected_channels[0]}', color='lightblue')
            ax1.plot(y_angles, y_ch2, 's-', label=f'Y-axis Ch{selected_channels[1]}', color='lightcoral')
        
        ax1.set_xlabel('Angle (degrees)')
        ax1.set_ylabel('Normalized Signal')
        ax1.set_title('Dual Axis Alignment Test')
        ax1.legend()
        ax1.grid(True)
        
        ax2 = fig.add_subplot(2, 1, 2)
        if x_indices:
            x_ratio = [ratio_values[i] for i in x_indices]
            ax2.plot(x_angles, x_ratio, 'o-', label='X-axis Ratio', color='green')
        if y_indices:
            y_ratio = [ratio_values[i] for i in y_indices]
            ax2.plot(y_angles, y_ratio, 's-', label='Y-axis Ratio', color='orange')
        
        ax2.set_xlabel('Angle (degrees)')
        ax2.set_ylabel('Ratio')
        ax2.set_title('Channel Ratios')
        ax2.legend()
        ax2.grid(True)
    
    fig.tight_layout()
    plt.draw()
    plt.pause(0.1)
    
    return fig

def read_serial_alignment_test(port='COM7', baudrate=230400):
    sampels_per_sec = 500
    pulse_detection_window = 1.0  # 1 second window to collect samples
    sampels_num = int(sampels_per_sec * pulse_detection_window)
    
    # Timing for display updates - every 1 second
    last_display_update = time.time()
    display_update_interval = 1.0  # Update display every 1 second
    
    # Get test configuration from user
    print("=== Optical Bench Alignment Test Setup ===")
    channels_input = input("Enter channels to monitor (e.g., 1,3): ")
    selected_channels = [int(x.strip()) for x in channels_input.split(',')]
    
    test_mode = input("Test mode - single axis (s) or dual axis (d)? ").lower()
    test_mode = "single" if test_mode == "s" else "dual"
    
    print("\n=== Setup Complete ===")
    print("1. Move to maximum power position for both X and Y")
    print("2. Press 'c' for calibration")
    print("3. Press 'g' to set starting angle")
    print("4. Move to starting position for first axis")
    print("5. Press 'a' to record measurements (angle auto-increments)")
    print("6. Press 's' to save and continue/finish")
    print("7. Press 'q' to quit")
    
    # Initialize variables
    c1, c2, c3, c4 = [], [], [], []
    ch_values = {1: [], 2: [], 3: [], 4: []}
    c1_med = c2_med = c3_med = c4_med = 0
    c1_display = c2_display = c3_display = c4_display = 0  # Values shown on screen
    
    # Peak detection for pulse synchronization
    c1_peak = c2_peak = c3_peak = c4_peak = 0
    peak_history = {1: [], 2: [], 3: [], 4: []}  # Store recent peaks
    
    # Test data storage
    angles = []
    measurements = {1: [], 2: [], 3: [], 4: []}
    ch1_norm = []
    ch2_norm = []
    ratio_values = []
    current_axis = []
    
    # Calibration values
    cal_ch1 = cal_ch2 = 1
    is_calibrated = False
    
    # Current measurement tracking
    measurement_count = 0
    current_test_axis = "X"
    current_angle = None  # Will be set when user presses 'g'
    angle_set = False     # Track if angle has been set
    
    # Initialize matplotlib
    plt.ion()
    plot_fig = plt.figure(figsize=(12, 8))
    plot_fig.show()
    
    ser = serial.Serial(port, baudrate, timeout=0.01)
    print(f"Connected to {port} at {baudrate} baud.")
    
    try:
        while True:
            black_image = np.zeros((height, width, 3), np.uint8)
            
            bytes_available = ser.in_waiting
            if bytes_available >= 16:
                data = ser.read(bytes_available)
                packets = [data[i:i+16] for i in range(len(data)-15) if data[i] == 0xFF]
                
                for packet in packets:
                    channels = process_packet(packet)
                    c1.append(channels[0])
                    c2.append(channels[1])
                    c3.append(channels[2])
                    c4.append(channels[3])
                    
                    # Track peak values for pulse detection
                    if channels[0] > c1_peak: c1_peak = channels[0]
                    if channels[1] > c2_peak: c2_peak = channels[1]
                    if channels[2] > c3_peak: c3_peak = channels[2]
                    if channels[3] > c4_peak: c4_peak = channels[3]
                    
                    # Process batch when enough samples collected
                    if len(c3) >= sampels_num:
                        c1, c2, c3, c4 = map(np.array, [c1, c2, c3, c4])
                        
                        # Get maximum value from the 1-second window (highest pulse peak) - NO CLIPPING
                        c1_max = np.max(c1) if len(c1) > 0 else 0
                        c2_max = np.max(c2) if len(c2) > 0 else 0
                        c3_max = np.max(c3) if len(c3) > 0 else 0
                        c4_max = np.max(c4) if len(c4) > 0 else 0
                        
                        # Set non-selected channels to 0 (offline)
                        if 1 not in selected_channels: c1_max = 0
                        if 2 not in selected_channels: c2_max = 0
                        if 3 not in selected_channels: c3_max = 0
                        if 4 not in selected_channels: c4_max = 0
                        
                        # For selected channels: filter out noise (below 100k) - NO SATURATION CAP, show real values
                        if 1 in selected_channels and c1_max < 100000: c1_max = 0
                        if 2 in selected_channels and c2_max < 100000: c2_max = 0
                        if 3 in selected_channels and c3_max < 100000: c3_max = 0
                        if 4 in selected_channels and c4_max < 100000: c4_max = 0
                        
                        # Calculate median for calibration (use max values since pulses are rare)
                        # For pulsed signals, median of mostly-zero data is meaningless, so use max
                        c1_med = c1_max
                        c2_med = c2_max
                        c3_med = c3_max
                        c4_med = c4_max
                        
                        # Update display values every 1 second
                        current_time = time.time()
                        if current_time - last_display_update >= display_update_interval:
                            # Display the maximum values from last 1-second window
                            c1_display = c1_max
                            c2_display = c2_max
                            c3_display = c3_max
                            c4_display = c4_max
                            
                            last_display_update = current_time
                        
                        c1, c2, c3, c4 = [], [], [], []
            
            # Create display
            black_image = np.zeros((height, width, 3), np.uint8)
            
            # Display format: gray for offline channels, green for active channels showing signal
            ch1_active = 1 in selected_channels
            ch2_active = 2 in selected_channels
            ch3_active = 3 in selected_channels
            ch4_active = 4 in selected_channels
            
            ch1_color = (0, 255, 0) if (ch1_active and c1_display > 0) else (100, 100, 100)
            ch2_color = (0, 255, 0) if (ch2_active and c2_display > 0) else (100, 100, 100)
            ch3_color = (0, 255, 0) if (ch3_active and c3_display > 0) else (100, 100, 100)
            ch4_color = (0, 255, 0) if (ch4_active and c4_display > 0) else (100, 100, 100)
            
            cv2.putText(black_image, f'Ch1: {np.round(c1_display/1e6,2)}M {"[OFFLINE]" if not ch1_active else ""}', (50, 50), font, 0.7, ch1_color, 2)
            cv2.putText(black_image, f'Ch2: {np.round(c2_display/1e6,2)}M {"[OFFLINE]" if not ch2_active else ""}', (50, 90), font, 0.7, ch2_color, 2)
            cv2.putText(black_image, f'Ch3: {np.round(c3_display/1e6,2)}M {"[OFFLINE]" if not ch3_active else ""}', (50, 130), font, 0.7, ch3_color, 2)
            cv2.putText(black_image, f'Ch4: {np.round(c4_display/1e6,2)}M {"[OFFLINE]" if not ch4_active else ""}', (50, 170), font, 0.7, ch4_color, 2)
            
            # Test info
            cv2.putText(black_image, f'Test: Ch{selected_channels[0]},Ch{selected_channels[1]}', (50, 220), font, 0.7, (255, 255, 0), 2)
            cv2.putText(black_image, f'Mode: {test_mode}', (50, 250), font, 0.7, (255, 255, 0), 2)
            cv2.putText(black_image, f'Measurements: {measurement_count}', (50, 280), font, 0.7, (255, 255, 0), 2)
            if current_angle is not None:
                cv2.putText(black_image, f'Current angle: {current_angle}', (50, 310), font, 0.7, (255, 255, 0), 2)
            if test_mode == "dual":
                cv2.putText(black_image, f'Current axis: {current_test_axis}', (50, 340), font, 0.7, (255, 255, 0), 2)
            
            # Calibration status
            cal_status = "CALIBRATED" if is_calibrated else "NOT CALIBRATED"
            cal_color = (0, 255, 0) if is_calibrated else (0, 0, 255)
            cv2.putText(black_image, cal_status, (50, 380), font, 0.7, cal_color, 2)
            
            # Instructions
            cv2.putText(black_image, "Keys:", (50, 420), font, 0.6, (255, 255, 255), 2)
            cv2.putText(black_image, "c - Calibrate", (50, 445), font, 0.5, (255, 255, 255), 1)
            cv2.putText(black_image, "g - Set starting angle", (50, 465), font, 0.5, (255, 255, 255), 1)
            cv2.putText(black_image, "a - Add measurement", (50, 485), font, 0.5, (255, 255, 255), 1)
            cv2.putText(black_image, "s - Save and continue", (50, 505), font, 0.5, (255, 255, 255), 1)
            cv2.putText(black_image, "q - Quit", (50, 525), font, 0.5, (255, 255, 255), 1)
            
            cv2.imshow('Optical Bench Alignment Test', black_image)
            key = cv2.waitKey(1) & 0xFF
            
            if key == ord('q'):
                break
                
            elif key == ord('c'):
                # Calibration mode
                ch1_val = [c1_med, c2_med, c3_med, c4_med][selected_channels[0]-1]
                ch2_val = [c1_med, c2_med, c3_med, c4_med][selected_channels[1]-1]
                
                if ch1_val > 0 and ch2_val > 0:
                    cal_ch1 = ch1_val
                    cal_ch2 = ch2_val
                    is_calibrated = True
                    print(f"Calibrated! Ch{selected_channels[0]}: {cal_ch1/1e6:.2f}M, Ch{selected_channels[1]}: {cal_ch2/1e6:.2f}M")
                else:
                    print("Cannot calibrate - one or both channels have zero values")
                    
            elif key == ord('g'):
                # Set starting angle
                print(f"\n--- Set Starting Angle ---")
                if test_mode == "dual":
                    print(f"Current axis: {current_test_axis}")
                
                try:
                    current_angle = float(input("Enter starting angle (degrees): "))
                    angle_set = True
                    print(f"Starting angle set to {current_angle}°. Press 'a' to add measurements (angle will auto-increment).")
                except ValueError:
                    print("Invalid angle entered. Please try again.")
                    
            elif key == ord('a'):
                # Add measurement
                if is_calibrated:
                    if not angle_set:
                        print("Please set starting angle first (press 'g')")
                        continue
                    
                    ch1_val = [c1_med, c2_med, c3_med, c4_med][selected_channels[0]-1]
                    ch2_val = [c1_med, c2_med, c3_med, c4_med][selected_channels[1]-1]
                    
                    # Store raw measurements
                    angles.append(current_angle)
                    measurements[1].append(c1_med)
                    measurements[2].append(c2_med)
                    measurements[3].append(c3_med)
                    measurements[4].append(c4_med)
                    
                    # Calculate normalized values
                    norm_ch1 = (ch1_val / cal_ch1) if cal_ch1 > 0 else 0
                    norm_ch2 = (ch2_val / cal_ch2) if cal_ch2 > 0 else 0
                    ch1_norm.append(norm_ch1)
                    ch2_norm.append(norm_ch2)
                    
                    # Calculate ratio
                    ratio = (norm_ch1 / norm_ch2) if norm_ch2 > 0 else 0
                    ratio_values.append(ratio)
                    
                    # Track axis for dual mode
                    current_axis.append(current_test_axis)
                    
                    measurement_count += 1
                    print(f"Measurement {measurement_count}: Angle {current_angle}°, Ch{selected_channels[0]} norm: {norm_ch1:.3f}, Ch{selected_channels[1]} norm: {norm_ch2:.3f}, Ratio: {ratio:.3f}")
                    
                    # Update live plot
                    plot_fig = update_live_plot(angles, ch1_norm, ch2_norm, ratio_values, selected_channels, test_mode, current_axis, plot_fig)
                    
                    # Auto-increment angle for next measurement
                    current_angle += 1
                else:
                    print("Please calibrate first (press 'c' at maximum power position)")
                    
            elif key == ord('s'):
                # Save measurements
                if len(angles) > 0:
                    if test_mode == "single":
                        # Single axis - save and finish
                        filename = input("Enter filename (without extension): ")
                        timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
                        save_path = rf"C:\Users\LAB3\Downloads\ir_bench\{filename}_{timestamp}.xlsx"
                        
                        # Create DataFrame
                        df_data = {
                            'angle': angles,
                            f'ch{selected_channels[0]}': [measurements[selected_channels[0]][i] for i in range(len(angles))],
                            f'ch{selected_channels[1]}': [measurements[selected_channels[1]][i] for i in range(len(angles))],
                            f'ch{selected_channels[0]}_normalized': ch1_norm,
                            f'ch{selected_channels[1]}_normalized': ch2_norm,
                            f'ratio_{selected_channels[0]}_{selected_channels[1]}': ratio_values
                        }
                        
                        df = pd.DataFrame(df_data)
                        df.to_excel(save_path, index=False)
                        print(f"Data saved to {save_path}")
                        
                        # Show final plot
                        plt.ioff()
                        update_live_plot(angles, ch1_norm, ch2_norm, ratio_values, selected_channels, test_mode, current_axis)
                        plt.show()
                        break
                        
                    else:
                        # Dual axis mode
                        if current_test_axis == "X":
                            # Finished X axis, start Y axis
                            print("X-axis measurements complete. Move to starting position for Y-axis test.")
                            input("Press Enter when ready to start Y-axis measurements...")
                            current_test_axis = "Y"
                            # Reset angle settings for Y-axis
                            current_angle = None
                            angle_set = False
                            print("Now set starting angle for Y-axis (press 'g'), then collect measurements with 'a'.")
                        else:
                            # Finished both axes - save and exit
                            filename = input("Enter filename (without extension): ")
                            timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
                            save_path = rf"C:\Users\LAB3\Downloads\ir_bench\{filename}_dual_axis_{timestamp}.xlsx"
                            
                            # Create DataFrame with axis information
                            df_data = {
                                'angle': angles,
                                'axis': current_axis,
                                f'ch{selected_channels[0]}': [measurements[selected_channels[0]][i] for i in range(len(angles))],
                                f'ch{selected_channels[1]}': [measurements[selected_channels[1]][i] for i in range(len(angles))],
                                f'ch{selected_channels[0]}_normalized': ch1_norm,
                                f'ch{selected_channels[1]}_normalized': ch2_norm,
                                f'ratio_{selected_channels[0]}_{selected_channels[1]}': ratio_values
                            }
                            
                            df = pd.DataFrame(df_data)
                            df.to_excel(save_path, index=False)
                            print(f"Data saved to {save_path}")
                            
                            # Show final plot
                            plt.ioff()
                            update_live_plot(angles, ch1_norm, ch2_norm, ratio_values, selected_channels, test_mode, current_axis)
                            plt.show()
                            break
                else:
                    print("No measurements to save!")
    
    except KeyboardInterrupt:
        print("Stopped by user.")
    finally:
        ser.close()
        cv2.destroyAllWindows()
        plt.close('all')

# Start the alignment test
read_serial_alignment_test(port='COM7', baudrate=230400)


=== Optical Bench Alignment Test Setup ===

=== Setup Complete ===
1. Move to maximum power position for both X and Y
2. Press 'c' for calibration
3. Press 'g' to set starting angle
4. Move to starting position for first axis
5. Press 'a' to record measurements (angle auto-increments)
6. Press 's' to save and continue/finish
7. Press 'q' to quit
Connected to COM7 at 230400 baud.

=== Setup Complete ===
1. Move to maximum power position for both X and Y
2. Press 'c' for calibration
3. Press 'g' to set starting angle
4. Move to starting position for first axis
5. Press 'a' to record measurements (angle auto-increments)
6. Press 's' to save and continue/finish
7. Press 'q' to quit
Connected to COM7 at 230400 baud.


C:\Users\thaim\AppData\Local\Temp\ipykernel_26320\4199198685.py:159: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plot_fig.show()
